## 台灣股市資訊模組：twstock

### 查詢歷史股票資料

In [ ]:
!pip install twstock

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import twstock

In [ ]:
stock = twstock.Stock('2317')

In [ ]:
print(stock.price)

[111.5, 111.5, 112.0, 111.5, 109.5, 109.0, 112.0, 109.5, 108.0, 110.0, 111.0, 112.0, 111.0, 109.0, 106.0, 100.0, 102.5, 100.5, 103.5, 102.0, 101.0, 100.5, 102.5, 105.0, 105.0, 105.0, 104.0, 103.5, 105.5, 106.0, 107.5]


In [ ]:
print(stock.price[-5:])

[104.0, 103.5, 105.5, 106.0, 107.5]


### 查詢即時股票資料

In [ ]:
real = twstock.realtime.get('2317')
real

{'info': {'channel': '2317.tw',
  'code': '2317',
  'fullname': '鴻海精密工業股份有限公司',
  'name': '鴻海',
  'time': '2022-07-26 02:51:35'},
 'realtime': {'accumulate_trade_volume': '10374',
  'best_ask_price': ['108.0000',
   '108.5000',
   '109.0000',
   '109.5000',
   '110.0000'],
  'best_ask_volume': ['4332', '5948', '4842', '5284', '5882'],
  'best_bid_price': ['107.5000',
   '107.0000',
   '106.5000',
   '106.0000',
   '105.5000'],
  'best_bid_volume': ['1932', '8490', '5480', '3236', '3153'],
  'high': '108.5000',
  'latest_trade_price': '108.0000',
  'low': '107.0000',
  'open': '108.0000',
  'trade_volume': '4'},
 'success': True,
 'timestamp': 1658803895.0}

In [ ]:
real['realtime']['latest_trade_price']

'108.0000'

In [ ]:
import twstock

stock = twstock.Stock('2317')  #鴻海
print('近31個收盤價：')
print(stock.price)   #近31個收盤價
print('近6個收盤價：')
print(stock.price[-6:])   #近6日之收盤價
while True:
    real = twstock.realtime.get('2317')
    if real['success']:
        if real['realtime']['latest_trade_price'] != '-':
            print('即時股票資料：')
            print(real)  #即時資料
            print('目前股價：')
            print(real['realtime']['latest_trade_price']) #即時價格
            break
    else:
        print('錯誤：' + real['rtmessage'])


近31個收盤價：
[111.5, 111.5, 112.0, 111.5, 109.5, 109.0, 112.0, 109.5, 108.0, 110.0, 111.0, 112.0, 111.0, 109.0, 106.0, 100.0, 102.5, 100.5, 103.5, 102.0, 101.0, 100.5, 102.5, 105.0, 105.0, 105.0, 104.0, 103.5, 105.5, 106.0, 107.5]
近6個收盤價：
[105.0, 104.0, 103.5, 105.5, 106.0, 107.5]
即時股票資料：
{'timestamp': 1658805629.0, 'info': {'code': '2317', 'channel': '2317.tw', 'name': '鴻海', 'fullname': '鴻海精密工業股份有限公司', 'time': '2022-07-26 03:20:29'}, 'realtime': {'latest_trade_price': '107.5000', 'trade_volume': '2', 'accumulate_trade_volume': '11213', 'best_bid_price': ['107.5000', '107.0000', '106.5000', '106.0000', '105.5000'], 'best_bid_volume': ['2145', '8486', '5927', '3233', '2990'], 'best_ask_price': ['108.0000', '108.5000', '109.0000', '109.5000', '110.0000'], 'best_ask_volume': ['4708', '6121', '4981', '5332', '5949'], 'open': '108.0000', 'high': '108.5000', 'low': '107.0000'}, 'success': True}
目前股價：
107.5000


## 發送 LINE Notify 通知

In [ ]:
import requests

msg = '這是 LINE Notify 發送的訊息。'
token = '輸入自己的權杖'  #權杖
headers = {
    "Authorization": "Bearer " + token, 
    "Content-Type" : "application/x-www-form-urlencoded"
}
payload = {'message': msg}
notify = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
if notify.status_code == 200:
    print('發送 LINE Notify 成功！')
else:
    print('發送 LINE Notify 失敗！')

發送 LINE Notify 成功！


## 實戰：用LINE傳送即時股價

In [ ]:
import twstock
import time
import requests

def lineNotify(token, msg):
    headers = {
        "Authorization": "Bearer " + token, 
        "Content-Type" : "application/x-www-form-urlencoded"
    }

    payload = {'message': msg}
    notify = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    return notify.status_code

def sendline(mode, realprice, counterLine, token):
    print('鴻海目前股價：' + str(realprice))
    if mode == 1:
        message = '現在鴻海股價為 ' + str(realprice) + '元，可以賣出股票了！'
    else:
        message = '現在鴻海股價為 ' + str(realprice) + '元，可以買入股票了！'
    code = lineNotify(token, message)
    if code == 200:
        counterLine = counterLine + 1
        print('第 ' + str(counterLine) + ' 次發送 LINE 訊息。')
    else:
        print('發送 LINE 訊息失敗！')
    return counterLine

token = '輸入自己的權杖'  #權杖
counterLine = 0  #儲存發送次數
counterError = 0  #儲存錯誤次數

print('程式開始執行！')
while True:
    realdata = twstock.realtime.get('2317')  #即時資料
    if realdata['success']:
        realprice = realdata['realtime']['latest_trade_price']  #目前股價
        if realprice != '-':
            if float(realprice) >= 80:
                counterLine = sendline(1, realprice, counterLine, token)
            elif float(realprice) <= 60:
                counterLine = sendline(2, realprice, counterLine, token)
            if counterLine >= 3:  #最多發送3次就結束程式
                print('程式結束！')
                break
            for i in range(300):  #每5分鐘讀一次
                time.sleep(1)       
    else:
        print('twstock 讀取錯誤，錯誤原因：' + realdata['rtmessage'])
        counterError = counterError + 1
        if counterError >= 3:  #最多錯誤3次
            print('程式結束！')
            break


程式開始執行！
鴻海目前股價：107.5000
第 1 次發送 LINE 訊息。
鴻海目前股價：107.5000
第 2 次發送 LINE 訊息。
鴻海目前股價：107.5000
第 3 次發送 LINE 訊息。
程式結束！
